<a href="https://colab.research.google.com/github/Tristan07999/TensorFlow2.0-with-tf.keras/blob/master/Movie_Reviews_(Text)_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#3RD SEP ZIQUAN LIN

#Using IMDB dataset contains the text of 50000 moive reviews 
#25000 reviews of training and 25000 reviews of testing
#For more advanced text classification, see MLCC text classification guide

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('version:', tf.__version__)
print('eager mode:', tf.executing_eagerly())
print('hub version:', hub.__version__)
print('GPU is', 'available' if tf.test.is_gpu_available() else 'Not availavle')


version: 2.0.0-rc0
eager mode: True
hub version: 0.5.0
GPU is Not availavle
